# NASA Earthdata API Client 🌍

CMR API documentation: https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html

EDL API documentation: https://urs.earthdata.nasa.gov/

## The Problem

### NASA Datasets and CMR

### Collections

### Granules

### UMM

### EDL

### Data Formats and Cloud Access

## A Python client for NASA's APIs


In [1]:
from earthdata import Auth, DataGranules, DataCollections, Accessor
# auth = Auth()

## Querying for collections
The DataCollection client can query CMR for any collection using all of CMR's Query parameters and has built-in accessors for the common ones.
This makes it ideal for one liners and easier notation.

```python
auth = Auth()
collections = DataCollections(auth).short_name('MODIS').get(10)
collections
```

We can filter fields, if we want the full UMM fileds we use a * symbol.

In [ ]:
# We can now search for collections using a pythonic API client for CMR.
Query = DataCollections(auth).keyword('fire').cloud_hosted(False)

print(f'Collections found: {Query.hits()}')

collections = Query.fields(['ShortName','Abstract']).get(100)
# Inspect 5 results printing just the ShortName and Abstract
collections[0:5]

The DataCollections class returns python dictionaries with some handy methods.

```python 
collection.concept_id() # returns the concept-id, used to search for data granules
collection.abstract() # returns the abstract
collection.landing_page() # returns the landing page if present in the UMM fields
collection.get_data() # returns the portal where data can be accessed.
```

The same results can be obtained using the `dict` syntax:

```python
collection["meta"]["concept-id"] # concept-id
collection["umm"]["RelatedUrls"] # URLs, with GET DATA, LANDING PAGE etc
```


In [ ]:
# We can now search for collections using a pythonic API client for CMR.
Query = DataCollections(auth).provider('POCLOUD')
print(f'Collections found: {Query.hits()}')
collections = Query.fields(['EntryTitle']).get(100)
# Printing 3 collections
collections[0:3]

In [ ]:
# Printing the concept-id for the first 10 collections
[collection.concept_id() for collection in collections[0:10]]

### Querying for data granules

The DataGranules class provides similar functionality as the collection class. To query for granules in a more reliable way concept-id would be the main key.
You can search data granules using a short name but that could (more likely will) return different versions of the same data granules. 

In this example we're querying for 20 data grnaules from ICESat-2 [ATL03](https://nsidc.org/data/ATL03/versions/) version `"003"` dataset. 

In [7]:
Query = DataGranules().short_name('ATL03').version("003")
granules = Query.get(20)

granules[0:3]

[Collection: {'EntryTitle': 'ATLAS/ICESat-2 L2A Global Geolocated Photon Data V003'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': -103.42874532191958, 'StartLatitude': -27.0, 'StartDirection': 'A', 'EndLatitude': 0.0, 'EndDirection': 'A'}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2018-10-13T23:56:45.392Z', 'EndingDateTime': '2018-10-14T00:03:47.795Z'}}
 Size(MB): 384.1767406464
 Data: ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL03.003/2018.10.13/ATL03_20181013235645_02340114_003_01.h5'],
 Collection: {'EntryTitle': 'ATLAS/ICESat-2 L2A Global Geolocated Photon Data V003'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Orbit': {'AscendingCrossing': -127.0482205607256, 'StartLatitude': 0.0, 'StartDirection': 'A', 'EndLatitude': 27.0, 'EndDirection': 'A'}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2018-10-14T00:03:47.797Z', 'EndingDateTime': '2018-10-14T00:10:49.722Z'}}
 Size(MB): 396.1719932556
 Data: ['https:/

### Spatiotemporal queries

Our granules and collection classes accept the same spatial and temporal argumenst as CMR so we can search for granules that match spatiotemporal criteria.



In [9]:
Query = DataGranules().short_name("ATL03").temporal("2020-03-01T01:02:00Z", "2020-03-30T00:00:30Z").bounding_box(-134.7,58.9,-133.9,59.2).version("003")
print(f"Granules found: {Query.hits()}")

Granules found: 4


In [10]:
# Now we can print some info about these granules using the built-in methods
granules = Query.get(4)
data_links = [{'links': g.data_links(), 'size (MB):': g.size()} for g in granules]
data_links

[{'links': ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL03.003/2020.03.06/ATL03_20200306122320_10810606_003_01.h5'],
  'size (MB):': 270.8442726135},
 {'links': ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL03.003/2020.03.08/ATL03_20200308234154_11190602_003_01.h5'],
  'size (MB):': 3333.9319086075},
 {'links': ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL03.003/2020.03.10/ATL03_20200310121504_11420606_003_01.h5'],
  'size (MB):': 170.8808164597},
 {'links': ['https://n5eil01u.ecs.nsidc.org/DP7/ATLAS/ATL03.003/2020.03.12/ATL03_20200312233336_11800602_003_01.h5'],
  'size (MB):': 2608.0768442154}]